In [1]:
# Autocompletion
%config Completer.use_jedi = False

# Autoreload  
%load_ext autoreload
%autoreload 2

In [2]:
# Autoreload
import sys
from typing import List

sys.path.append('../')

import numpy as np
from IPython.display import Audio, display


import nltk
import torch
import numpy as np
import datasets
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM

In [3]:
device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

from llmsearch.utils.mem_utils import gc_cuda

print(f"Device - {device}")

def beep(duration = 1, frequency=440, rhythm=1):
    sample_rate = 44100  # Standard audio sample rate
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    audio_data = np.sin(2*np.pi*frequency*t)  # Generate a sine wave
    audio_data *= np.where(np.arange(len(audio_data)) % rhythm == 0, 1, 0)  # Apply rhythm
    display(Audio(audio_data, rate=sample_rate, autoplay=True))

Device - mps


In [4]:
dataset = datasets.load_dataset("samsum")

Found cached dataset samsum (/Users/praful932/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
sample_size = 100
samples_to_tune_on = datasets.Dataset.from_dict(dataset["train"][:sample_size])
samples_to_tune_on = samples_to_tune_on.rename_columns(column_mapping = {'dialogue' : 'X', 'summary' : "y"})

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast = False)
model =  AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)



In [7]:
import langchain

X = samples_to_tune_on[0]['X']

pt = langchain.PromptTemplate.from_template("Conversation: {X}\nSummary:")

pt.format_prompt(X = X).to_string()

"Conversation: Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)\nSummary:"

In [8]:
import evaluate

rouge_metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def get_rouge_score(y_true: List, y_pred: List):
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True)
    return result['rouge2']

In [9]:
from llmsearch.tuner import Tuner
from llmsearch.utils.mem_utils import get_total_available_ram, get_gpu_information

from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

seed = 42

total_available_ram_memory = get_total_available_ram()

print(f"Total available ram memory - {total_available_ram_memory}\n")

tuner_ob = Tuner(model = model,tokenizer = tokenizer,dataset = samples_to_tune_on,
                 device = device, batch_size = 512,
                 tokenizer_encoding_kwargs={'padding': True, 'truncation': True, 'max_length': 512},
                 tokenizer_decoding_kwargs = {'skip_special_tokens' : True,  'spaces_between_special_tokens' : False}, 
                 scorer = get_rouge_score, prompt_template = pt, is_encoder_decoder = True, seed = seed, column_mapping = {"text_column_name": "X", "label_column_name": "y"})


Total available ram memory - 8



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Initializing new estimator with generation keys - {}




In [10]:
# Earlier
from llmsearch.utils.model_utils import seed_everything


"""
parameters and how they affect do_sample == False
1. temperature - output does not change - greedy decoding
2. top_k - output does not change - greedy decoding
3. repetition_penalty - output changes
4. no_repeat_ngram_size - output changes
"""

# seed_everything(seed)

initial_generation_params1 = {
    'max_new_tokens' : 120,
}
# score, outputs1 = tuner_ob.get_score(initial_generation_params1)

In [11]:
hyp_param_grid = {
    "max_new_tokens": [120],
    "temperature": list(np.linspace(start=0.1, stop=1.0,num=10)),
    'top_k' : list(map(int,np.linspace(start=10, stop=50,num=5))),
    "top_p": [0.75, 0.8, 0.9, 1.0],
    'do_sample' : [True, False],
    'generation_seed' : [42],
    'repetition_penalty' : [1.0, 1.2],
    'no_repeat_ngram_size' : [0,2,3],
}

hyp_param_grid_2= {
    "max_new_tokens": [120],
    "temperature": list(np.linspace(start=0.1, stop=1.0,num=10000)),
    'top_k' : list(map(int,np.linspace(start=10, stop=50,num=5000))),
    "top_p": list(map(int,np.linspace(start=10, stop=50,num=5000))),
    'do_sample' : [True],
    'generation_seed' : [42],
#     'repetition_penalty' : [1.0, 1.2],
#     'no_repeat_ngram_size' : [0,2,3],
}

scorer = make_scorer(score_func=get_rouge_score, greater_is_better=True)


clf = RandomizedSearchCV(
    estimator=tuner_ob.estimator,
    param_distributions=hyp_param_grid_2,
    n_iter = 2,
    scoring=scorer,
    cv=5,
    random_state = 42,
    n_jobs=1,
    verbose=3,
)

"""
5 fold means, whole sample set of 100 examples will be split into 80:20 ratio
for each hyper_parameter set we have a model f(hyper_params)
    - we will evaluate this model and get the cross val score (test on each 20 samples 5 times, while training on the rest 80 each time)
    - we get the score on the quality of hyperparams by evaluating the model with the hyperparams on the unseen 1 fold
"""


'\n5 fold means, whole sample set of 100 examples will be split into 80:20 ratio\nfor each hyper_parameter set we have a model f(hyper_params)\n    - we will evaluate this model and get the cross val score (test on each 20 samples 5 times, while training on the rest 80 each time)\n    - we get the score on the quality of hyperparams by evaluating the model with the hyperparams on the unseen 1 fold\n'

In [12]:
len(tuner_ob.dataset['X'])

100

In [13]:
def foo(a):
    for item in a:
        print(f"input to function is {a}")
        foo()

In [14]:
clf.fit(X=tuner_ob.dataset["X"], y=tuner_ob.dataset["y"])

Before cloning
{} 


Fitting 5 folds for each of 2 candidates, totalling 10 fits
Before cloning
{} 


After setting
{'top_p': 36, 'top_k': 17, 'temperature': 0.6999099909990999, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


Performing inference with batch_size - 512
Performing inference using - {'top_p': 36, 'top_k': 17, 'temperature': 0.6999099909990999, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True}




  0%|          | 0/1 [00:00<?, ?it/s]

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:719: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:2671: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:


Total available ram memory - 6, Total available gpu memory - None

[CV 1/5] END do_sample=True, generation_seed=42, max_new_tokens=120, temperature=0.6999099909990999, top_k=17, top_p=36;, score=0.172 total time=  14.0s
Before cloning
{'top_p': 36, 'top_k': 17, 'temperature': 0.6999099909990999, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


After setting
{'top_p': 36, 'top_k': 17, 'temperature': 0.6999099909990999, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


Performing inference with batch_size - 512
Performing inference using - {'top_p': 36, 'top_k': 17, 'temperature': 0.6999099909990999, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True}




  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Total available ram memory - 5, Total available gpu memory - None

[CV 4/5] END do_sample=True, generation_seed=42, max_new_tokens=120, temperature=0.5357335733573357, top_k=45, top_p=15;, score=0.179 total time=   3.1s
Before cloning
{'top_p': 15, 'top_k': 45, 'temperature': 0.5357335733573357, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


After setting
{'top_p': 15, 'top_k': 45, 'temperature': 0.5357335733573357, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


Performing inference with batch_size - 512
Performing inference using - {'top_p': 15, 'top_k': 45, 'temperature': 0.5357335733573357, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True}




  0%|          | 0/1 [00:00<?, ?it/s]

Total available ram memory - 5, Total available gpu memory - None

[CV 5/5] END do_sample=True, generation_seed=42, max_new_tokens=120, temperature=0.5357335733573357, top_k=45, top_p=15;, score=0.209 total time=   3.0s
Before cloning
{'top_p': 15, 'top_k': 45, 'temperature': 0.5357335733573357, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


After setting
{'top_p': 15, 'top_k': 45, 'temperature': 0.5357335733573357, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 


Before cloning
{'top_p': 15, 'top_k': 45, 'temperature': 0.5357335733573357, 'max_new_tokens': 120, 'generation_seed': 42, 'do_sample': True} 




RandomizedSearchCV(cv=5,
                   estimator=LLMEstimatorWrapper(batch_size=512, device='mps',
                                                 disable_batch_size_cache=False,
                                                 do_sample=True,
                                                 generation_seed=42,
                                                 is_encoder_decoder=True,
                                                 is_fitted_=True,
                                                 max_new_tokens=120,
                                                 model=T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Bloc...
                                                        0.10234023402340235,
                                                        0.10243024302430244,
                                                        0.10252025202520253,
                                                        0.10261026102610261, ...],
                                        'top_k': [10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, ...],
                                        'top_p': [10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, 10, 10, 10, 10, 10,
                                                  10, 10, ...]},
                   random_state=42, scoring=make_scorer(get_rouge_score),
                   verbose=3)

In [15]:
clf.best_params_

{'top_p': 15,
 'top_k': 45,
 'temperature': 0.5357335733573357,
 'max_new_tokens': 120,
 'generation_seed': 42,
 'do_sample': True}

In [24]:
clf.best_estimator_

EstimatorWrapper(batch_size=512, device='mps', disable_batch_size_cache=False,
                 do_sample=True, generation_seed=42, is_encoder_decoder=True,
                 is_fitted_=True, max_new_tokens=120,
                 model=T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfA...
                 tokenizer=T5Tokenizer(name_or_path='google/flan-t5-small', vocab_size=32100, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_id_57>', '<extra_id_58>', '<extra_id_59>', '<extra_id_60>', '<extra_id_61>', '<extra_id_62>', '<extra_id_63>', '<extra_id_64>', '<extra_id_65>', '<extra_id_66>', '<extra_id_67>', '<extra_id_68>', '<extra_id_69>', '<extra_id_70>', '<extra_id_71>', '<extra_id_72>', '<extra_id_73>', '<extra_id_74>', '<extra_id_75>', '<extra_id_76>', '<extra_id_77>', '<extra_id_78>', '<extra_id_79>', '<extra_id_80>', '<extra_id_81>', '<extra_id_82>', '<extra_id_83>', '<extra_id_84>', '<extra_id_85>', '<extra_id_86>', '<extra_id_87>', '<extra_id_88>', '<extra_id_89>', '<extra_id_90>', '<extra_id_91>', '<extra_id_92>', '<extra_id_93>', '<extra_id_94>', '<extra_id_95>', '<extra_id_96>', '<extra_id_97>', '<extra_id_98>', '<extra_id_99>']}, clean_up_tokenization_spaces=True),
                 tokenizer_decoding_kwargs={'skip_special_tokens': True,
                                            'spaces_between_special_tokens': False},
                 tokenizer_encoding_kwargs={'max_length': 512, 'padding': True,
                                            'truncation': True},
                 top_k=10, top_p=0.75)

In [25]:
clf.best_estimator_.set_params(**clf.best_params_).get_params()


{'model': T5ForConditionalGeneration(
   (shared): Embedding(32128, 512)
   (encoder): T5Stack(
     (embed_tokens): Embedding(32128, 512)
     (block): ModuleList(
       (0): T5Block(
         (layer): ModuleList(
           (0): T5LayerSelfAttention(
             (SelfAttention): T5Attention(
               (q): Linear(in_features=512, out_features=384, bias=False)
               (k): Linear(in_features=512, out_features=384, bias=False)
               (v): Linear(in_features=512, out_features=384, bias=False)
               (o): Linear(in_features=384, out_features=512, bias=False)
               (relative_attention_bias): Embedding(32, 6)
             )
             (layer_norm): T5LayerNorm()
             (dropout): Dropout(p=0.1, inplace=False)
           )
           (1): T5LayerFF(
             (DenseReluDense): T5DenseGatedActDense(
               (wi_0): Linear(in_features=512, out_features=1024, bias=False)
               (wi_1): Linear(in_features=512, out_features=1024, b

In [ ]:
clf.fi

In [ ]:
from sklearn.base import clone

clone(1)

In [ ]:
dir(clf)

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_estimator_.get_params()